In [25]:
import gurobipy as gp
from gurobipy import Model, GRB
import pandas as pd
import numpy as np

In [26]:
df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 2 Files\BasketballPlayers.csv") 

In [27]:
df

,Number,Position,Ball Handling,Shooting,Rebounding,Defense,Athletic Ability,Toughness,Mental Acuity
0,1,G/F,1,2,3,2,1,2,1
1,2,G/F,1,1,1,2,3,2,3
2,3,G/F,3,1,1,2,3,2,1
3,4,G/F,2,3,2,2,2,1,1
4,5,F/C,1,2,3,3,3,3,2
...,...,...,...,...,...,...,...,...,...
145,146,F/C,2,2,3,1,1,3,3
146,147,G/F,2,3,2,3,2,2,2
147,148,G/F,3,1,2,3,2,3,3
148,149,F/C,1,2,3,1,1,2,2


In [28]:
m = gp.Model("BasketballTeamSelection")

In [29]:
# Add decision variables

players = df.index.tolist()
player_vars = m.addVars(players, vtype=GRB.BINARY, name="Player")

In [30]:
# Constraints

# Guard and Forward/Center allocation constraints
guards = df[df['Position'].isin(['G', 'G/F'])].index
forward_centers = df[df['Position'].isin(['F', 'C', 'F/C'])].index
m.addConstr(player_vars.sum(guards) >= 0.3 * 21, "MinGuards")
m.addConstr(player_vars.sum(forward_centers) >= 0.4 * 21, "MinForwardCenters")

<gurobi.Constr *Awaiting Model Update*>

In [31]:
# Skill average constraints

skills = ['Ball Handling', 'Shooting', 'Rebounding', 'Defense', 'Athletic Ability', 'Toughness', 'Mental Acuity']

for skill in skills:
    total_skill_points = gp.quicksum(player_vars[i] * df.at[i, skill] for i in players)
    m.addConstr(total_skill_points >= 2.05 * 21, f"Skill_{skill}_Adjusted")

In [32]:
# Exclusive invitation constraints

group1 = list(range(20, 25))
group2 = list(range(72, 79))
group3 = list(range(105, 115))
group4 = list(range(45, 50))
group5 = list(range(65, 70))
m.addConstr(gp.quicksum(player_vars[i] for i in group1) * gp.quicksum(player_vars[i] for i in group2) == 0, "Exclusive1")
m.addConstr(gp.quicksum(player_vars[i] for i in group3) <= (gp.quicksum(player_vars[i] for i in group4) + gp.quicksum(player_vars[i] for i in group5)), "DependentInvitations")

<gurobi.Constr *Awaiting Model Update*>

In [33]:
# At least one player from each range constraint

for start in range(1, 141, 10):
    end = start + 9
    m.addConstr(gp.quicksum(player_vars[i] for i in range(start, end+1)) >= 1, f"AtLeastOne_{start}-{end}")

In [34]:
m.addConstr(player_vars.sum() == 21, "Exactly21Invitations")

<gurobi.Constr *Awaiting Model Update*>

In [35]:
m.setObjective(gp.quicksum(player_vars[i] * df.loc[i, skills].sum() for i in players), GRB.MAXIMIZE)

In [36]:
# Optimize the model

m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 25 rows, 150 columns and 1510 nonzeros
Model fingerprint: 0x7b25a2fb
Model has 1 quadratic constraint
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve added 35 rows and 0 columns
Presolve time: 0.00s
Presolved: 60 rows, 150 columns, 1241 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)

Root relaxation: objective 3.620000e+02, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

In [37]:
# Extract the solution

selected_players = [i for i in players if player_vars[i].X > 0.5]
print("Selected Players:", selected_players)

Selected Players: [4, 6, 15, 25, 36, 46, 55, 69, 73, 75, 89, 94, 103, 109, 110, 117, 130, 131, 132, 133, 140]


In [38]:
len(selected_players)

21

In [39]:
optimal_value = m.objVal

print(f"Optimal Objective Function Value: {optimal_value}")

Optimal Objective Function Value: 362.0


In [40]:
num_guards_invited = sum(player_vars[i].X for i in guards)
print(f"Number of Guards Invited: {num_guards_invited}")

Number of Guards Invited: 10.0


In [45]:
def adjust_and_solve_model(min_invitations):
    
    m = gp.Model("BasketballTeamSelection")
    
    # Define variables
    players = df.index.tolist()
    player_vars = m.addVars(players, vtype=GRB.BINARY, name="Player")
    
    # Guards and Forward/Center allocation constraints based on current 'min_invitations'
    guards = df[df['Position'].isin(['G', 'G/F'])].index
    forward_centers = df[df['Position'].isin(['F', 'C', 'F/C'])].index
    m.addConstr(player_vars.sum(guards) >= 0.3 * min_invitations, "MinGuards")
    m.addConstr(player_vars.sum(forward_centers) >= 0.4 * min_invitations, "MinForwardCenters")
    
    # Skill average constraints adjusted for 'min_invitations'
    for skill in skills:
        total_skill_points = gp.quicksum(player_vars[i] * df.at[i, skill] for i in players)
        m.addConstr(total_skill_points >= 2.05 * min_invitations, f"Skill_{skill}_Adjusted")
    
    # Exclusive invitation and dependent invitations constraints
    m.addConstr(gp.quicksum(player_vars[i] for i in group1) * gp.quicksum(player_vars[i] for i in group2) == 0, "Exclusive1")
    m.addConstr(gp.quicksum(player_vars[i] for i in group3) <= (gp.quicksum(player_vars[i] for i in group4) + gp.quicksum(player_vars[i] for i in group5)), "DependentInvitations")

    for i in range(1, 151, 10):
        m.addConstr(sum(x[j] for j in players.index if i <= players.loc[j, 'Number'] < i + 10) >= 1, f"Group_{i}_{i+9}")
    
    # Set objective
    m.setObjective(gp.quicksum(player_vars[i] * df.loc[i, skills].sum() for i in players), GRB.MAXIMIZE)
    
    # Solve the model
    m.optimize()
    
    return m

In [42]:
min_invitations = 21

In [43]:
while min_invitations > 0:
    model = adjust_and_solve_model(min_invitations)
    if model.status == GRB.INFEASIBLE:
        print(f"Model becomes infeasible with {min_invitations} invitations.")
        model.computeIIS()  # Identify infeasible constraints
        # Print out all constraints that are part of the IIS
        for c in model.getConstrs():
            if c.IISConstr:
                print(f"Infeasible constraint: {c.constrName}")
    else:
        min_invitations -= 1  # Decrease and try again

if min_invitations == 0:
    print("Model did not become infeasible with any number of invitations. Check constraints and model setup.")

TypeError: 'builtin_function_or_method' object is not iterable

In [51]:
players = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 2 Files\BasketballPlayers.csv") 

players['Average'] = players.iloc[:, 2:].mean(axis=1)

# Filter players with average skill above 2.05 (without resetting the index)
filtered_players = players[players['Average'] > 2.05]
filtered_players = filtered_players.drop(columns=['Average'])

# Number of players
num_players = len(filtered_players)

# Create the optimization model
m = Model("BasketballTeamSelection")

# Create binary decision variables for each player (using original indices)
x = m.addVars(filtered_players.index, vtype=GRB.BINARY, name="Player")

# Pre-compute the player positions using original indices
guards = [i for i in filtered_players.index if filtered_players.loc[i, 'Position'] in ['G', 'G/F']]
forwards_centers = [i for i in filtered_players.index if filtered_players.loc[i, 'Position'] in ['F', 'C', 'F/C']]

# Total number of players selected
total_players_selected = sum(x[i] for i in filtered_players.index)    
    
# At least 30% of the invitations should go to guards
m.addConstr(player_vars.sum(guards) >= 0.3 * min_invitations, "MinGuards")

# At least 40% of the invitations should go to forwards/centers
m.addConstr(player_vars.sum(forward_centers) >= 0.4 * min_invitations, "MinForwardCenters")

# If any player from 20-24 (inclusive) is invited, all players from 72-78 (inclusive) cannot be, and vice versa
m.addConstr(sum(x[i] for i in filtered_players.index if 20 <= filtered_players.loc[i, 'Number'] <= 24) + sum(x[j] for j in filtered_players.index if 72 <= filtered_players.loc[j, 'Number'] <= 78) <= 1, "Group_20_24_vs_72_78")


# If any player from 105-114 (inclusive) is invited, at least one player from 45-49 (inclusive) and 65-69 (inclusive) must be invited
for i in [idx for idx in filtered_players.index if 105 <= filtered_players.loc[idx, 'Number'] <= 114]:
    m.addConstr(x[i] <= sum(x[j] for j in filtered_players.index if 45 <= filtered_players.loc[j, 'Number'] <= 49) + sum(x[k] for k in filtered_players.index if 65 <= filtered_players.loc[k, 'Number'] <= 69), f"Group_105_114_requires_{i}")


# At least one player must be invited from: 1-10, 11-20, 21-30, ..., 131-140, 141-150
for i in range(1, 151, 10):
    m.addConstr(sum(x[j] for j in filtered_players.index if i <= filtered_players.loc[j, 'Number'] < i + 10) >= 1, f"Group_{i}_{i+9}")

# Update the model
m.update()

# Print the number of constraints in the model
print("Number of constraints after adding:", len(m.getConstrs()))

# Check if there are any constraints in the model
constraints = m.getConstrs()
if not constraints:
    raise ValueError("No constraints found in the model")
last_feasible_solution = constraints[0]

# Change the objective function to minimize the total number of players selected
m.setObjective(total_players_selected, GRB.MINIMIZE)

# Find the smallest number of players that can be selected without causing infeasibility
min_players_selected = num_players
infeasible_constraint = None

for i in range(num_players, 0, -1):
    m.update()
    m.optimize()

    # Find the selected players
    selected_players = [i for i in filtered_players.index if x[i].X > 0.5]

# Print the details of the selected players
    print("Selected players:")
    for player in selected_players:
        player_data = filtered_players.loc[player]
        print(f"Player Number: {player_data['Number']}, Position: {player_data['Position']}")

    if m.status == GRB.INFEASIBLE:
        infeasible_constraint = m.getConstrs()[m.getConstrs().index(last_feasible_solution) + 1].ConstrName
        break
    elif m.status == GRB.OPTIMAL:
        min_players_selected = i
        # Update last_feasible_solution only if it's not the last constraint
        if m.getConstrs().index(last_feasible_solution) + 1 < len(m.getConstrs()):
            last_feasible_solution = m.getConstrs()[m.getConstrs().index(last_feasible_solution) + 1]

print(f"Value of the objective function (total number of players selected): {m.ObjVal}")
if infeasible_constraint:
    print(f"The constraint that caused infeasibility: {infeasible_constraint}")

m.addConstr(total_players_selected <= m.ObjVal - 1, "Reduced_Player_Selection")
m.optimize()
if m.status == GRB.INFEASIBLE:
    print("Model is infeasible. Identifying the problematic constraint(s)...")
    m.computeIIS()
    for c in m.getConstrs():
        if c.IISConstr:
            print(f"Constraint causing infeasibility: {c.ConstrName}")

GurobiError: Variable not in model